# Demonstration of xQTL protocols

To perform a complete analysis from molecular phenotype calling up to the xqtl discovery as demonstrated in the xqtl-protocol paper(in preparation), please read the ***mini protocols*** in the following orders:

## Molecular phenotype calling

1. [Reference data procession](https://cumc.github.io/xqtl-pipeline/code/data_preprocessing/reference_data.html)
2. [Gene expression calling for eQTL](https://cumc.github.io/xqtl-pipeline/code/molecular_phenotypes/bulk_expression.html)
3. [Alternative splicing calling for sQTL](https://cumc.github.io/xqtl-pipeline/code/molecular_phenotypes/splicing.html)

## xQTL discovery

1. [Phenotype Processing](https://cumc.github.io/xqtl-pipeline/code/data_preprocessing/phenotype_preprocessing.html)
2. [Genotype Processing](https://cumc.github.io/xqtl-pipeline/code/data_preprocessing/genotype_preprocessing.html)
3. [Covariates Processing](https://cumc.github.io/xqtl-pipeline/code/data_preprocessing/covariate_preprocessing.html)
4. [CisQTL Association Scan](https://cumc.github.io/xqtl-pipeline/code/association_scan/cisQTL_scan.html)
5. [TransQTL Association Scan](https://cumc.github.io/xqtl-pipeline/code/association_scan/transQeL_scan.html)